# **BANK CUSTOMER CHURN PREDICTION**
![Photo by <a href="https://unsplash.com/@clemono?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Clem Onojeghuo</a> on <a href="https://unsplash.com/s/photos/churn?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  ](https://images.unsplash.com/photo-1465125159090-b104362936e1?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1190&q=80)
> Image credits: Clem Onojeghuo from Unsplash

Churn prediction models have become a very important asset in determining the customer behaviour for a company. The churn rate drives decision making and makes the company analyse itself and the way they provide its services to the customer. Churn prediction consists of detecting which customers are likely to cancel a subscription to a service based on how they use the service. As it is more costlier to get new customers than to keep the old ones, Churn models have become very necessary.
<br/>
For a bank, predicting the churn rate is to determine which customers are closing their bank accounts and what factors are leading to their decisions. A Churn Prediction Model might be useful in this case to predict way before a person might close their account. The factors leading to customer churn are also very important to analyse closely and make important decisions in changing or modifying the current workings of the company or a service provider.

>To create a churn model I have selected the `churn-modelling` dataset. The dataset consists of various information on the customers and a target column called `'Exited'` that tells whether they have closed their bank accounts. Let's start!

# Contents:
1. <a href='#imports'>Imports</a>
2. <a href='#dataanalysis'>Data Analysis</a>
   1. <a href='#datareading'>Data Reading</a>
   2. <a href='#datacleaning'>Data Cleaning</a>
   3. <a href='#datawrangling'>Data Wrangling with EDA</a>
3. <a href='#modeltraining'>Model Training</a>
   1. <a href='#lrmodel'>Training a Logistic Regression model as a base model</a>
   2. <a href='#xgboost'>Training on XG-Boost</a>
   3. <a href='#hyperparam'>Hyperparameter tuning for XG-Boost</a>
4. <a href='#results'>Results</a>

# 1. <a id='imports'>Imports</a>
### Let us start by importing our usual libraries.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from numpy import mean

import warnings
warnings.filterwarnings('ignore')

seed = 46

# 2. <a id='dataanalysis'>Data Analysis</a>
### We will start by analysing the data. First we will read the dataset and see for any missing values using pandas. Then we will go through the dataset and replace all the values which are not in the right format.

## 2.1. <a id='datareading'>Data Reading</a>

In [ ]:
data = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')

In [ ]:
len(data)

In [ ]:
data.head()

In [ ]:
data.tail()

>#### Creating a copy so that we can revert back to the original dataset whenever necessary.

In [ ]:
df = data.copy()

## 2.2. <a id='datacleaning'>Data Cleaning</a>
### Let's first see if the target classes have equal number of entries or not.

In [ ]:
df['Exited'].value_counts()

<a id="imb"></a>

In [ ]:
print('Class imbalance percentage: {}%'.format(len(df[df['Exited']==1])/(len(df[df['Exited']==0]))*100))

> #### The class 0 has around 6000 more values than the class 1 making this dataset skewed and the classes imbalanced.

In [ ]:
print(df.columns)

In [ ]:
df.isnull().sum()

>#### No missing values found so we move on to the next step.

## 2.3. <a id='datawrangling'>Data Wrangling with EDA</a>
### Here, we will conduct some Exploratory Data Analysis to determine which features are necessary and which are just useless. We will also convert categorical features into numerical ones to make it easier for the model to understand our data.

> #### Features: CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary

> #### Target: Exited

#### Heatmap for our data:

In [ ]:
plt.figure(figsize=(12,12))
churn_corr = df.corr()
matrix = np.triu(churn_corr)
churn_corr_top = churn_corr.index
sns.heatmap(df[churn_corr_top].corr(), annot=True, vmin=1.0, vmax=-1.0, mask=matrix, cmap="cool", alpha=0.55)

#### Class Imbalance shown visually:

In [ ]:
plt.figure(figsize=(15, 3))
sns.countplot(y=df['Exited'], data=df, palette="cool", alpha=0.55, saturation=1)

### `Credit Scores`:
#### Distribution plot for credit scores:

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(x=data["CreditScore"], color="violet", axlabel="Credit Scores")
plt.show()

>#### Here I have divided the Credit score column in 5 parts by using `pd.cut()`. The table below shows the 5 bands and the exiting value mean for those bands.

In [ ]:
df['CreditScoreBand'] = pd.cut(df['CreditScore'], 5)
df[['CreditScoreBand', 'Exited']].groupby(['CreditScoreBand'], as_index=False).mean().sort_values(by='CreditScoreBand', ascending=True)

In [ ]:
df_ = [df] #making a list so that 'for loops' can be easily implemented

>#### Here what I have done is that I have replaced each Credit score band with a value (from 0 to 4) so that now only these values will be displayed instead of the actual credit scores in the table. 

In [ ]:
for dataset in df_:    
    dataset.loc[(dataset['CreditScore'] > 349.5) & (dataset['CreditScore'] <= 450.0), 'CreditScore'] = 0
    dataset.loc[(dataset['CreditScore'] > 450.0) & (dataset['CreditScore'] <= 550.0), 'CreditScore'] = 1
    dataset.loc[(dataset['CreditScore'] > 550.0) & (dataset['CreditScore'] <= 650.0), 'CreditScore'] = 2
    dataset.loc[(dataset['CreditScore'] > 650.0) & (dataset['CreditScore'] <= 750.0), 'CreditScore'] = 3
    dataset.loc[(dataset['CreditScore'] > 750.0) & (dataset['CreditScore'] <= 850.0), 'CreditScore'] = 4
df.head()

>#### Now, let's drop the CreditScoreBand column which has now done its purpose.

In [ ]:
df = df.drop(['CreditScoreBand'], axis=1)
df_ = [df]

#### Thus, we have replaced the continuous distribution of the Credit scores into 5 discrete values. Similarly we will conduct such modifications on all the continuous features.

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.distplot(x=data["CreditScore"], color="violet", axlabel="Credit Scores", ax=ax[0])
sns.distplot(x=df["CreditScore"], color="violet", axlabel="Credit Scores", ax=ax[1])
fig.set_figheight(5)
fig.set_figwidth(15)
plt.show()

### `Geography`
#### Plot for Geography with Exited:

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(y="Geography", hue="Exited", data=data, palette="cool", alpha=0.55)
plt.show()

#### Here, we can conduct One Hot Encoding and replace the categories: France, Germany and Spain into numbers. But, I went on a more raw approach.

In [ ]:
for dataset in df_:
    dataset['Geography'] = dataset['Geography'].map( {'France': 0, 'Germany': 1, 'Spain': 2} ).astype(int)

df.head()

### `Gender`:
#### Plot for Gender:

In [ ]:
plt.figure(figsize=(15,3))
sns.countplot(y=df['Gender'], orient="v", data=df, palette="cool", alpha=0.55)
plt.show()

#### Exiting rate based on the gender:

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(y="Exited", hue="Gender", data=data, orient="v", palette="cool", alpha=0.55)
plt.show()

In [ ]:
for dataset in df_:
    dataset['Gender'] = dataset['Gender'].map( {'Female': 1, 'Male': 0} ).astype(int)
df.head()

### `Age`:
#### Distribution plot for age of the customers:

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(x=data["Age"], bins=30, axlabel="Age", color='violet')
plt.show()

In [ ]:
df['AgeBand'] = pd.cut(df['Age'], 5)
df[['AgeBand', 'Exited']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in df_:    
    dataset.loc[(dataset['Age'] > 17.926) & (dataset['Age'] <= 32.8), 'Age'] = 0
    dataset.loc[(dataset['Age'] > 32.8) & (dataset['Age'] <= 47.6), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 47.6) & (dataset['Age'] <= 62.4), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 62.4) & (dataset['Age'] <= 77.2), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 77.2, 'Age'] = 4
df.head()

In [ ]:
df = df.drop(['AgeBand'], axis=1)
df.head()

### `Tenure`:
>#### Tenure has 11 unique values that are just number of years from 0 to 10. Hence, I kept it as it is.

In [ ]:
df['Tenure'].nunique()

In [ ]:
df['Tenure'].unique()

### `Bank Balance`:
#### Distribution plot for customer bank balance:

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(x=data["Balance"], axlabel="Bank Balance", color="violet")
plt.show()

In [ ]:
df['BalanceBand'] = pd.cut(df['Balance'], 5)
df[['BalanceBand', 'Exited']].groupby(['BalanceBand'], as_index=False).mean().sort_values(by='BalanceBand', ascending=True)

In [ ]:
df_ = [df]

In [ ]:
for dataset in df_:    
    dataset.loc[dataset['Balance'] <= 50179.618, 'Balance'] = 0
    dataset.loc[(dataset['Balance'] > 50179.618) & (dataset['Balance'] <= 100359.236), 'Balance'] = 1
    dataset.loc[(dataset['Balance'] > 100359.236) & (dataset['Balance'] <= 150538.854), 'Balance'] = 2
    dataset.loc[(dataset['Balance'] > 150538.854) & (dataset['Balance'] <= 200718.472), 'Balance'] = 3
    dataset.loc[(dataset['Balance'] > 200718.472) & (dataset['Balance'] <=  250898.09), 'Balance'] = 4
    df['Balance'] = df['Balance'].astype(int)
df.head()

In [ ]:
df = df.drop(['BalanceBand'], axis=1)
df_ = [df]

### `Estimated Salary`:
#### Distribution plot for customer salary:

In [ ]:
plt.figure(figsize=(12, 5))
sns.distplot(x=data["EstimatedSalary"], color="violet")
plt.show()

In [ ]:
df['EstimatedSalaryBand'] = pd.cut(df['EstimatedSalary'], 4)
df[['EstimatedSalaryBand', 'Exited']].groupby(['EstimatedSalaryBand'], as_index=False).mean().sort_values(by='EstimatedSalaryBand', ascending=True)

In [ ]:
for dataset in df_:    
    dataset.loc[(dataset['EstimatedSalary'] > -188.401) & (dataset['EstimatedSalary'] <= 50006.805), 'EstimatedSalary'] = 0
    dataset.loc[(dataset['EstimatedSalary'] > 50006.805) & (dataset['EstimatedSalary'] <= 100002.03), 'EstimatedSalary'] = 1
    dataset.loc[(dataset['EstimatedSalary'] > 100002.03) & (dataset['EstimatedSalary'] <= 149997.255), 'EstimatedSalary'] = 2
    dataset.loc[(dataset['EstimatedSalary'] > 149997.255) & (dataset['EstimatedSalary'] <= 199992.48), 'EstimatedSalary'] = 3
    df['EstimatedSalary'] = df['EstimatedSalary'].astype(int)
df.head()

In [ ]:
df = df.drop(['EstimatedSalaryBand'], axis=1)
df.head()

>#### Removing other columns with very little to no information. 

In [ ]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
df.head()

#### Here, we have a modified table with all the values converted into a format that our model can understand. Now, let's get it prepared to pass it into the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, data['Exited'], test_size=0.15, random_state=seed)

# 3. <a id='modeltraining'>Model Training</a>
## 3.1. <a id='lrmodel'>Treating a Logistic Regression model as a base model</a>

In [ ]:
LR_model = LogisticRegression(random_state=seed)
LR_model.fit(X_train, y_train)

In [ ]:
y_pred = LR_model.predict(X_test)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)
fig.set_figheight(5)
fig.set_figwidth(12)
ax1.set_title('ROC Curve')
ax2.set_title('Precision Recall Curve')
plot_roc_curve(LR_model, X_test, y_test, ax=ax1)
plot_precision_recall_curve(LR_model, X_test, y_test, ax=ax2)

In [ ]:
print(classification_report(y_test, y_pred))

## 3.2. <a id='xgboost'>Training on XG-Boost</a>
>#### Calling the XG-Boost model object and setting `tree_method='gpu_hist'` to be able to train the model on the GPU which is much faster than training it normally on CPU.

In [ ]:
XGB_model = XGBClassifier(tree_method='gpu_hist')
XGB_model.fit(X_train, y_train)
print('Your model is trained!')

In [ ]:
y_pred1 = XGB_model.predict(X_test)
fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)
fig.set_figheight(5)
fig.set_figwidth(12)
ax1.set_title('ROC Curve')
ax2.set_title('Precision Recall Curve')
plot_roc_curve(XGB_model, X_test, y_test, ax=ax1)
plot_precision_recall_curve(XGB_model, X_test, y_test, ax=ax2)

In [ ]:
print(classification_report(y_test, y_pred1))

## 3.3. <a id='hyperparam'>HyperParameter Tuning</a>
#### Determining the best hyperparameters using GridSearchCV

>`scale_pos_weight`: Control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: sum(negative instances) / sum(positive instances) ==> already calculated over<a href="#imb"> here.</a>  

>`alpha`: L1 regularization term on weights. Increasing this value will make model more conservative.  

>`max_depth`: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.  

>`gamma`: Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger `gamma` is, the more conservative the algorithm will be. 

>`subsample`: Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. 

>`learning_rate`: Step size shrinkage used in update to prevent overfitting. 

>`tree_method`: Basically to train XGBoost on cpu or gpu. Read more over in the <a href="https://xgboost.readthedocs.io/en/latest/gpu/index.html?highlight=tree_method">documentation.</a>

>`eval_metric`: Evaluation metrics for validation data, a default metric will be assigned according to objective. 

>`objective: 'binary:hinge'`: hinge loss for binary classification. This makes predictions of 0 or 1, rather than producing probabilities. *I tried using this but the score went below 72%.*  

>`objective: 'binary:logistic'`: logistic regression for binary classification, output probability.



In [ ]:
MAX_TREE_DEPTH = [4]#complex>>>
TREE_METHOD = ['gpu_hist']
SUBSAMPLE = [0.7]
REGULARIZATION = [0.1, 0.01]
GAMMA = [0.2, 0.3]
POS_WEIGHT = [0.26]
OBJ = ['binary:logistic']
EVAL = ['auc']
ETA = [0.11]

param_grid = {'tree_method': TREE_METHOD, 'max_depth': MAX_TREE_DEPTH, 'alpha': REGULARIZATION,
          'gamma': GAMMA, 'subsample': SUBSAMPLE, 'scale_pos_weight': POS_WEIGHT, 'learning_rate': ETA, 
          'objective': OBJ, 'eval_metric': EVAL}

>#### The above parameters are what I found to give the greatest score (by trial and error).

### Introducing Cross Validation:

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)

In [ ]:
%%time
grid = GridSearchCV(estimator=XGB_model, param_grid=param_grid, scoring='roc_auc', cv=cv, n_jobs=-1)
grid_fit = grid.fit(X_train,y_train)

>### The training process has been completed!!

# 4. <a id="results">Results</a>
### Thus, I have successfully trained the XG Boost model using cross validation and finally fine tuned the hyperparameters, with the best scores as shown below.

In [ ]:
best_score = grid_fit.best_score_
best_params = grid_fit.best_params_

print('Best Score: {:.2f} % with these parameters: {})'.format(best_score*100, best_params))

### Other parameters which I tried:
- Best Score: **85.40%** with these parameters: {'alpha': 0.01, 'eval_metric': 'auc', 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 0.26, 'subsample': 0.5, 'tree_method': 'gpu_hist'}), 
- Best Score: **85.43%** with these parameters: {'alpha': 0.01, 'eval_metric': 'auc', 'gamma': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 0.26, 'subsample': 0.6, 'tree_method': 'gpu_hist'}),
- Best Score: **85.44%** with these parameters: {'alpha': 0.01, 'eval_metric': 'auc', 'gamma': 0.2, 'learning_rate': 0.11, 'max_depth': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 0.26, 'subsample': 0.7, 'tree_method': 'gpu_hist'}),  
- Best Score: **85.44%** with these parameters: {'alpha': 0.1, 'eval_metric': 'auc', 'gamma': 0.3, 'learning_rate': 0.11, 'max_depth': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 0.26, 'subsample': 0.7, 'tree_method': 'gpu_hist'})


In [ ]:
#Passing our final model with the test data
y_pred2 = grid_fit.predict(X_test)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)
fig.set_figheight(5)
fig.set_figwidth(12)
ax1.set_title('ROC Curve')
ax2.set_title('Precision Recall Curve')
plot_roc_curve(grid_fit, X_test, y_test, ax=ax1)
plot_precision_recall_curve(grid_fit, X_test, y_test, ax=ax2)
print('Classification Report: \n', classification_report(y_test, y_pred2))

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_figheight(7.5)
fig.set_figwidth(18)
fig.suptitle('ROC Curve for Different models')

#ROC curves for Training set
ax1.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.6)
ax1.set_title("Train Data Results")
plot_roc_curve(LR_model, X_train, y_train, ax=ax1)
plot_roc_curve(XGB_model, X_train, y_train, ax=ax1)
plot_roc_curve(grid_fit, X_train, y_train, ax=ax1)

#ROC curves for Testing set
ax2.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.6)
ax2.set_title("Test Data Results")
plot_roc_curve(LR_model, X_test, y_test, ax=ax2)
plot_roc_curve(XGB_model, X_test, y_test, ax=ax2)
plot_roc_curve(grid_fit, X_test, y_test, ax=ax2)

#### The above two plots give the ROC AUC curve for the training set and for the testing set. It can be observed that the curve for training set is more smooth as compared to the curves for the testing sets.
#### For the XG-Boost model trained **without** cross validation and hyperparameter tuning, the training AUC score is **0.95** but while testing it falls to **0.81**, suggesting that the model must have overfit to the data. 
#### For the XG-Boost model which used cross validation during training and was hyper tuned, the training score was **0.88** and the testing score was **0.84**, suggesting that the model did not overfit to the training data and hence the testing score was the highest among all three models.
#### Overall, the Hypertuned XG-Boost model performs the best among other models on testing set.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_figheight(7.5)
fig.set_figwidth(18)
fig.suptitle('Precision Recall Curve for Different models')

#ROC curves for Training set
ax1.set_title("Train Data Results")
plot_precision_recall_curve(LR_model, X_train, y_train, ax=ax1)
plot_precision_recall_curve(XGB_model, X_train, y_train, ax=ax1)
plot_precision_recall_curve(grid_fit, X_train, y_train, ax=ax1)

#ROC curves for Testing set
ax2.set_title("Test Data Results")
plot_precision_recall_curve(LR_model, X_test, y_test, ax=ax2)
plot_precision_recall_curve(XGB_model, X_test, y_test, ax=ax2)
plot_precision_recall_curve(grid_fit, X_test, y_test, ax=ax2)

#### An excerpt from [this](http://acutecaretesting.org/en/articles/precision-recall-curves-what-are-they-and-how-are-they-used) article referenced:  
>#### The example presented above clearly shows that an imbalanced data set with a large fraction of persons without disease in the test population will make a ROC curve look better than it would in a balanced data set with fewer persons without disease.   

#### This suggests that the ROC curves automatically shows better performance when the dataset is imbalanced as compared to when the dataset in not imbalanced. Hence, relying just on ROCs would not be good for understanding the model performance.  
#### The above two figures show Precision Recall Curves for training and testing datasets on different models. The curve above the other curve has a better performance level.  
#### First figure shows the XGBClassifier model as performing the best but it is clearly overfitting to the data as seen in the second figure where the GridSearchCV i.e. the hypertuned and cross-validated model comes on top.

In [ ]:

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3)
fig.set_figheight(5)
fig.set_figwidth(20)
fig.suptitle('Confusion Matrices for Different models')

ax1.set_title('Logistic Regression')
LR_conf = confusion_matrix(y_test, y_pred)
sns.heatmap(LR_conf, annot=True, fmt='d', ax=ax1, linewidths=3, linecolor='white')

ax2.set_title('XG-Boost Model')
XG_conf_1 = confusion_matrix(y_test, y_pred1)
sns.heatmap(XG_conf_1, annot=True, fmt='d', ax=ax2, linewidths=3, linecolor='white')

ax3.set_title('Hypertuned XG-Boost Model')
XG_conf_2 = confusion_matrix(y_test, y_pred2)
sns.heatmap(XG_conf_2, annot=True, fmt='d', ax=ax3, linewidths=3, linecolor='white')

print('Total no. of testing samples: ', len(y_test))

### Thus, we can say after looking at the ROC and PRC charts that the performance improved substantially when XG-Boost was paired with fine tuning and cross validation. 
### Though the overall performance was not that great we can still improve it by using more advanced methods such as synthetic data creation to remove the imbalance in the dataset and see if the performance improves further.

# Referenced:
[1] https://machinelearningmastery.com/xgboost-for-imbalanced-classification/  
[2] https://xgboost.readthedocs.io/en/latest/parameter.html?highlight=learning%20rate#learning-task-parameters  
[3] https://acutecaretesting.org/en/articles/precision-recall-curves-what-are-they-and-how-are-they-used